In [184]:
import numpy as np
import pandas as pd 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.model_selection import cross_val_score

In [235]:
data = pd.read_csv("NLP_train.csv")
test = pd.read_csv("NLP_test.csv")

In [236]:
target = data.target
data.drop(["location","keyword","id","target"],axis = 1,inplace = True)

In [237]:
len(target)

7613

In [240]:
##Data Cleaning
import re
import string

In [241]:
def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
round1 = lambda x: clean_text_round1(x)


In [242]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data.text.apply(round1))
data_clean.iloc[3],data.iloc[3]

(text     people receive wildfires evacuation orders in...
 Name: 3, dtype: object,
 text    13,000 people receive #wildfires evacuation or...
 Name: 3, dtype: object)

In [243]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [244]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.text.apply(round2))
data_clean.iloc[3],data.iloc[3]

(text     people receive wildfires evacuation orders in...
 Name: 3, dtype: object,
 text    13,000 people receive #wildfires evacuation or...
 Name: 3, dtype: object)

In [245]:
len(words)

235892

In [253]:
##removing none-english words from the data
words = set(nltk.corpus.words.words())
for idx,i in enumerate(data_clean.text):
    text = nltk.wordpunct_tokenize(i)
    for j in text:
        if j not in words:
            data_clean.text[idx] = data_clean.text[idx].replace(f"{j}","  ")
        

In [254]:
data_clean

,text
0,our are the reason of this earthquake may ...
1,forest fire near la canada
2,all to shelter in place are being notifi...
3,people receive evacuation in
4,just got sent this photo from ruby as smoke...
...,...
7608,two giant holding a bridge collapse into ne...
7609,the out of control wild in even in...
7610,s of volcano
7611,police investigating after an with a car...


Document-Term Matrix

In [255]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

C:\Users\amoha\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aba,abandon,abandoned,ability,abject,ablaze,able,aboard,abomination,...,zar,zeal,zero,zimbabwe,zip,zipper,zodiac,zombie,zone,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [256]:
a = []
for i in data_dtm.columns:
    if i not in words:
        a.append(i)

In [257]:
for i in a:
    data_dtm.drop([f"{i}"], inplace = True, axis = 1)

In [258]:
data_dtm

,aa,aba,abandon,abandoned,ability,abject,ablaze,able,aboard,abomination,...,zar,zeal,zero,zimbabwe,zip,zipper,zodiac,zombie,zone,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [259]:
data_dtm["text"]

0       0
1       0
2       0
3       0
4       0
       ..
7608    0
7609    0
7610    0
7611    0
7612    0
Name: text, Length: 7613, dtype: int64

In [262]:
from gensim import matutils, models
import scipy.sparse

In [265]:
tdm = data_dtm.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,7603,7604,7605,7606,7607,7608,7609,7610,7611,7612
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aba,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abandon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abandoned,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [266]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [267]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

NameError: name 'pickle' is not defined

In [219]:
target

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [228]:
data_dtm["target"] = target

,aa,aba,abandon,abandoned,ability,abject,ablaze,able,aboard,abomination,...,zar,zeal,zero,zimbabwe,zip,zipper,zodiac,zombie,zone,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [222]:
train_vectors

<6140x6140 sparse matrix of type '<class 'numpy.int64'>'
	with 6140 stored elements in Compressed Sparse Row format>

In [215]:
train_vectors = count_vectorizer.fit_transform(data_dtm)


In [217]:
test_vectors = count_vectorizer.transform(test["text"])

In [213]:
count_vectorizer = feature_extraction.text.CountVectorizer()

In [212]:
train_vectors = count_vectorizer.fit_transform(data["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.

test_vectors = count_vectorizer.transform(test["text"])
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, train_vectors, data["target"], cv=3, scoring="f1")

In [18]:
scores

array([0.59453669, 0.5642787 , 0.64117647])

In [23]:
!python --version

Python 3.9.7


In [26]:
!pip install --upgrade scikit.learn

  Attempting uninstall: scikit.learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\amoha\\anaconda3\\Lib\\site-packages\\~klearn\\feature_extraction\\_hashing_fast.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

